In [2]:
from osman import Osman, OsmanConfig
import pandas as pd

In [3]:
OS_HOST_URL="http://localhost:9200"
TEMPLATE_NAME = 'semantic_search'
INDEX_NAME = "bible"

In [4]:
os_man = Osman(OsmanConfig(host_url=OS_HOST_URL))

In [5]:
source = {
    "_source": {
        "excludes": ["passage_embedding"]
    },
    "query": {
        "bool": {
            "should": [
                {
                    "fuzzy": {
                        "content": {  
                            "value": "{{query_text}}",
                            "fuzziness": 2,  
                            "prefix_length": 1  
                        }
                    }
                }
            ],
            "minimum_should_match": 1  
        }
    }
}

params = {
    "query_text": "christ"  # Example search text
}

os_man.upload_search_template(
    source=source, 
    name=TEMPLATE_NAME, 
    index=INDEX_NAME, 
    params=params
)


{'acknowledged': False}

In [6]:
from opensearchpy import OpenSearch

# Connection settings
host = 'localhost'
port = 9200

# Connect to OpenSearch
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

In [14]:
# Define the template parameters
template_params = {
    "id": TEMPLATE_NAME,  # The name of the template you uploaded
    "params": {
        "query_text": "hamburger"  # The text to search for
    }
}

# Execute the search using the search template
response = client.search_template(
    body=template_params,
    index=INDEX_NAME  # The name of the index where the template should be applied
)

# Print the response
print(response)


{'took': 19, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


In [15]:
# Extracting the data from the response
hits = response['hits']['hits']
data = [
    {
        "Index": hit['_index'],
        "Score": hit['_score'],
        "Verse ID": hit['_source']['verse_id'],
        "Content": hit['_source']['content'].replace('\n', ' ')  # Replacing newline characters for better display
    }
    for hit in hits
]

# Creating a DataFrame
df = pd.DataFrame(data)

In [16]:
df

""
